# L4: JSON Game Mechanics

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.<br>
<span style="font-size: larger;">To maintain consistency, the notebooks are run with a 'world state' consistent with the video at the start of each notebook.</span></p>

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

## Define Inventory Detector

In [1]:
system_prompt = """You are an AI Game Assistant. \
Your job is to detect changes to a player's \
inventory based on the most recent story and game state.
If a player picks up, or gains an item add it to the inventory \
with a positive change_amount.
If a player loses an item remove it from their inventory \
with a negative change_amount.
Given a player name, inventory and story, return a list of json update
of the player's inventory in the following form.
Only take items that it's clear the player (you) lost.
Only give items that it's clear the player gained. 
Don't make any other item updates.
If no items were changed return {"itemUpdates": []}
and nothing else.

Response must be in Valid JSON
Don't add items that were already added in the inventory

Inventory Updates:
{
    "itemUpdates": [
        {"name": <ITEM NAME>, 
        "change_amount": <CHANGE AMOUNT>}...
    ]
}
"""


In [2]:
import json
# from helper import get_together_api_key, load_env
# from together import Together

# client = Together(api_key=get_together_api_key())
from groq import Groq
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())

client = Groq(api_key=os.getenv("GROQ_API_KEY"))
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": "hello world who are you"}],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")

Hello. I'm an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."

In [3]:
def detect_inventory_changes(game_state, output):
    inventory = game_state["inventory"]
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Current Inventory: {str(inventory)}"},
        {"role": "user", "content": f"Recent Story: {output}"},
        {"role": "user", "content": "Inventory Updates"},
    ]
    chat_completion = client.chat.completions.create(
        # response_format={"type": "json_object", "schema": InventoryUpdate.model_json_schema()},
        model="llama-3.3-70b-versatile",
        temperature=0.0,
        messages=messages,
    )
    response = chat_completion.choices[0].message.content
    result = json.loads(response)
    return result["itemUpdates"]

In [4]:
from helper import get_game_state

game_state = get_game_state()
game_state["inventory"] = {
    "cloth pants": 1,
}

result = detect_inventory_changes(
    game_state, "You buy a sword from the merchant for 5 gold"
)

print(result)

/Users/ob1/projects/aisg-agent-exploration/gaming/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You are Kaelin Darkhaven, a young and skilled horse trainer from Brindlemark. You stand at the entrance of the famous Brindlemark Stables, where you've spent countless hours training and raising exotic horses. The sound of neighing horses and the smell of fresh hay fill the air as you gaze out at the rolling hills and lush pastures beyond the stables. The annual Skyrunner's Cup is just around the corner, and you've been working tirelessly to prepare your prized stallion, Starlight, for the competition. As you look out at the stables, you notice a commotion near the town square, with people gathered around a large poster that's just been put up.
game_state loaded from json:  {'world': "Kyrethia is a world where cities are built on the backs of massive beasts known as the Eldrid, ancient creatures that roam the land and have been domesticated by its inhabitants, each city is a unique fusion of architecture and biology, with the Eldrid's movements and behaviors influencing the daily life 

In [5]:
def update_inventory(inventory, item_updates):
    update_msg = ""

    for update in item_updates:
        name = update["name"]
        change_amount = update["change_amount"]

        if change_amount > 0:
            if name not in inventory:
                inventory[name] = change_amount
            else:
                inventory[name] += change_amount
            update_msg += f"\nInventory: {name} +{change_amount}"
        elif name in inventory and change_amount < 0:
            inventory[name] += change_amount
            update_msg += f"\nInventory: {name} {change_amount}"

        if name in inventory and inventory[name] < 0:
            del inventory[name]

    return update_msg


#### Now include inventory in the story

In [6]:
# def run_action(message, history, game_state):

#     if(message == 'start game'):
#         return game_state['start']

#     system_prompt = """You are an AI Game master. Your job is to write what \
# happens next in a player's adventure game.\
# Instructions: \
# You must on only write 1-3 sentences in response. \
# Always write in second person present tense. \
# Ex. (You look north and see...) \
# Don't let the player use items they don't have in their inventory.
# """

#     world_info = f"""
# World: {game_state['world']}
# Kingdom: {game_state['kingdom']}
# Town: {game_state['town']}
# Your Character:  {game_state['character']}
# Inventory: {json.dumps(game_state['inventory'])}"""

#     messages = [
#         {"role": "system", "content": system_prompt},
#         {"role": "user", "content": world_info}
#     ]

#     for action in history:
#         messages.append({"role": "assistant", "content": action[0]})
#         messages.append({"role": "user", "content": action[1]})

#     messages.append({"role": "user", "content": message})
#     client = Together(api_key=get_together_api_key())
#     model_output = client.chat.completions.create(
#         model="meta-llama/Llama-3-70b-chat-hf",
#         messages=messages
#     )

#     result = model_output.choices[0].message.content
#     return result

## Integrate into the Game

In [7]:
from helper import start_game, get_game_state

game_state = get_game_state(
    inventory={
        "cloth pants": 1,
        "cloth shirt": 1,
        "goggles": 1,
        "leather bound journal": 1,
        "gold": 5,
    }
)

You are Kaelin Darkhaven, a young and skilled horse trainer from Brindlemark. You stand at the entrance of the famous Brindlemark Stables, where you've spent countless hours training and raising exotic horses. The sound of neighing horses and the smell of fresh hay fill the air as you gaze out at the rolling hills and lush pastures beyond the stables. The annual Skyrunner's Cup is just around the corner, and you've been working tirelessly to prepare your prized stallion, Starlight, for the competition. As you look out at the stables, you notice a commotion near the town square, with people gathered around a large poster that's just been put up.
game_state loaded from json:  {'world': "Kyrethia is a world where cities are built on the backs of massive beasts known as the Eldrid, ancient creatures that roam the land and have been domesticated by its inhabitants, each city is a unique fusion of architecture and biology, with the Eldrid's movements and behaviors influencing the daily life 

In [8]:
from helper import run_action, is_safe


def main_loop(message, history):
    output = run_action(message, history, game_state)

    safe = is_safe(output)
    if not safe:
        return "Invalid Output"

    item_updates = detect_inventory_changes(game_state, output)
    update_msg = update_inventory(game_state["inventory"], item_updates)
    output += update_msg

    return output


start_game(main_loop, True)

ERROR:    [Errno 48] error while attempting to bind on address ('0.0.0.0', 7860): address already in use


Running on local URL:  http://0.0.0.0:7861
Running on public URL: https://5c69f038acb489ca8a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


appending action:  ['start game', "You are Kaelin Darkhaven, a young and skilled horse trainer from Brindlemark. You stand at the entrance of the famous Brindlemark Stables, where you've spent countless hours training and raising exotic horses. The sound of neighing horses and the smell of fresh hay fill the air as you gaze out at the rolling hills and lush pastures beyond the stables. The annual Skyrunner's Cup is just around the corner, and you've been working tirelessly to prepare your prized stallion, Starlight, for the competition. As you look out at the stables, you notice a commotion near the town square, with people gathered around a large poster that's just been put up."]


Traceback (most recent call last):
  File "/Users/ob1/projects/aisg-agent-exploration/gaming/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ob1/projects/aisg-agent-exploration/gaming/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ob1/projects/aisg-agent-exploration/gaming/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ob1/projects/aisg-agent-exploration/gaming/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1518, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ob1/projects/aisg

appending action:  ['Look around', "You look around and see the bustling town of Brindlemark, with its colorful stalls and marketplaces, nestled between the scaly ridges of the smaller Eldrid's back, the sound of horse whinnies and the smell of fresh hay filling the air. People of all ages hurry about, preparing for the upcoming Skyrunner's Cup, with trainers and breeders alike eager to showcase their finest horses. You notice the Brindlemark Stables, where you spent most of your days training and caring for the horses, standing proudly at the town's center, its doors open wide in anticipation of the big event."]
appending action:  ['Look around', "You look around and see the bustling town of Brindlemark, with its colorful stalls and marketplaces, nestled between the scaly ridges of the smaller Eldrid's back, the sound of horse whinnies and the smell of fresh hay filling the air. People of all ages hurry about, preparing for the upcoming Skyrunner's Cup, with trainers and breeders alik